## Prepping GZ2 Manifest
This notebook will load in the full GZ:2 Manifest and extract a some random non-interacting and interacting images. We will then use a script to download the selected images into a new folder and then prepare them for upload to the HEC for classification. Will keep an 'answered' and non-answered .csv file so that there's no chance of any cheating. Can then compare what went right and wrong in the output of the model.predict() part.

In [7]:
import pandas as pd
from sklearn.utils import shuffle
import os

In [2]:
folder = 'C:/Users/oryan/Documents/zoobot/sdss-predict-sample/'

In [4]:
df = pd.read_csv(folder+'full-gz2-manifest.csv',index_col=0)

In [8]:
len(df)

239695

In [6]:
df[:10]

,ra,dec,rastring,decstring,sample,gz2_class,total_classifications,total_votes,t01_smooth_or_features_a01_smooth_count,t01_smooth_or_features_a01_smooth_weight,...,t11_arms_number_a36_more_than_4_fraction,t11_arms_number_a36_more_than_4_weighted_fraction,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a36_more_than_4_flag,t11_arms_number_a37_cant_tell_count,t11_arms_number_a37_cant_tell_weight,t11_arms_number_a37_cant_tell_fraction,t11_arms_number_a37_cant_tell_weighted_fraction,t11_arms_number_a37_cant_tell_debiased,t11_arms_number_a37_cant_tell_flag
dr7objid,,,,,,,,,,,,,,,,,,,,,
587732591714893851,179.042984,60.522518,11:56:10.32,+60:31:21.1,original,Sc+t,45,342,0,0.0,...,0.450,0.450,0.482646,0,16,16.0,0.400,0.400,0.394506,0
588009368545984617,135.084396,52.494240,09:00:20.26,+52:29:39.3,original,Sb+t,42,332,1,1.0,...,0.512,0.503,0.504833,0,13,13.0,0.317,0.323,0.322743,0
587732484359913515,183.371979,50.741508,12:13:29.27,+50:44:29.4,original,Ei,36,125,28,28.0,...,0.000,0.000,0.000000,0,0,0.0,0.000,0.000,0.000000,0
587741723357282317,186.251953,28.558598,12:25:00.47,+28:33:31.0,original,Sc+t,28,218,1,1.0,...,0.240,0.240,0.241322,0,6,6.0,0.240,0.240,0.239765,0
587738410866966577,161.086395,14.084465,10:44:20.73,+14:05:04.1,original,Er,43,151,33,33.0,...,0.000,0.000,0.000000,0,0,0.0,0.000,0.000,0.000000,0
587729751132209314,246.921387,40.926968,16:27:41.13,+40:55:37.1,extra,Ei,48,154,41,41.0,...,0.000,0.000,0.000000,0,0,0.0,0.000,0.000,0.000000,0
587733608555216981,249.474640,36.073040,16:37:53.91,+36:04:22.9,original,Ei,39,142,25,25.0,...,0.000,0.000,0.000000,0,0,0.0,0.000,0.000,0.000000,0
587735742617616406,243.146759,29.365067,16:12:35.22,+29:21:54.2,original,Sb+t,35,282,0,0.0,...,0.457,0.442,0.480484,0,8,8.0,0.229,0.235,0.230543,0
587738574068908121,195.278030,39.841473,13:01:06.73,+39:50:29.3,original,Ei,50,158,42,42.0,...,0.000,0.000,0.000000,0,0,0.0,0.000,0.000,0.000000,0


In [7]:
df_clsf_red = df.query('total_classifications >= 20')

In [11]:
df_clsf_red['t08_odd_feature_a24_merger_flag'].value_counts()

0    237520
1      2159
Name: t08_odd_feature_a24_merger_flag, dtype: int64

In [12]:
df_clsf_merg_tmp = df_clsf_red[['ra','dec','t08_odd_feature_a24_merger_flag']]

In [13]:
df_clsf_merg = df_clsf_merg_tmp.query('t08_odd_feature_a24_merger_flag == 1')
df_clsf_no_merg = df_clsf_merg_tmp.query('t08_odd_feature_a24_merger_flag == 0')

In [15]:
red_df_clsf_merg = df_clsf_merg.sample(200)
red_df_clsf_no_merg = df_clsf_no_merg.sample(800)

In [18]:
df_manifest_uniform = pd.concat([red_df_clsf_merg, red_df_clsf_no_merg])

In [26]:
df_manifest = (
    shuffle(df_manifest_uniform)
    .reset_index()
)

In [27]:
df_manifest_unknown = (
    df_manifest
    .drop(columns='t08_odd_feature_a24_merger_flag')
)

In [28]:
df_manifest.to_csv('C:/Users/oryan/Documents/zoobot/sdss-predict-sample/zoobot-pred-manifest-known.csv')

In [29]:
df_manifest_unknown.to_csv('C:/Users/oryan/Documents/zoobot/sdss-predict-sample/zoobot-pred-manifest-unknown.csv')

### Remaking the locations

In [5]:
df_loaded = pd.read_csv('C:/Users/oryan/Documents/zoobot/sdss-predict-sample/combined-sample-manifest.csv',index_col = 0)

In [10]:
os.path.basename(df_loaded.filepath.iloc[0])

'587729159514751105.png'

In [12]:
df_hec_paths = df_loaded.assign(filepath_HEC = df_loaded.filepath.apply(lambda x: '/mmfs1/home/users/oryan/Zoobot/data/gz2_cutouts/'+os.path.basename(x)))

In [14]:
df_hec_paths.filepath_HEC.iloc[0]

'/mmfs1/home/users/oryan/Zoobot/data/gz2_cutouts/587729159514751105.png'

In [16]:
df_hec_paths

,dr7objid,ra,dec,filepath,filepath_HEC
0,587729159514751105,220.943970,4.407396,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
1,587739844308369546,214.318024,21.509171,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
2,587726032800448671,214.308960,2.202336,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
3,587741602567553201,181.391235,28.052105,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
4,587725576426946785,262.621704,56.164040,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
...,...,...,...,...,...
995,587742903403151521,196.944351,17.479954,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
996,588016878288437515,122.758492,22.480146,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
997,588010359080484975,166.815781,4.690566,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...
998,588011218071191794,237.644348,48.814133,C:\Users\oryan\Documents\zoobot\sdss-predict-s...,/mmfs1/home/users/oryan/Zoobot/data/gz2_cutout...


In [17]:
df_hec_export = (
    df_hec_paths
    .drop(columns='filepath')
    .rename(columns={'filepath_HEC':'filepath'})
)

In [19]:
df_hec_export.to_csv('C:/Users/oryan/Documents/zoobot/sdss-predict-sample/test-pred-gz2-manifest.csv')